In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import seaborn as sns
import os
import os.path as osp
import sys
import pickle
import joblib
from collections import Counter
from itertools import product
import torch
import pdb
import random
import tables
from sklearn.linear_model import LogisticRegression, LinearRegression
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, average_precision_score,\
balanced_accuracy_score
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.calibration import CalibratedClassifierCV
import wandb
from wandb.lightgbm import wandb_callback, log_summary
from dill.source import getsource
from dill import detect
import functools

### set the seeds and change to current directory + set the output directory

In [3]:
SEED=90210
np.random.seed(SEED)
os.environ['USER_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/'
os.environ['OUT_PATH']='/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/output_directory'

### Create function to pickle functions

In [4]:
def function_to_string(fn):
    return getsource(detect.code(fn)) 


In [5]:
processed_data_path=osp.join(os.environ.get('OUT_PATH'), 'ParksInspection')


In [6]:
os.environ['WANDB_SILENT']="true"


In [7]:
os.chdir('/share/pierson/selective_labels_data/hirid_data_analysis/richras_dir/learning_from_doctor_and_patient/')
from AnalysisFuncs import plotCorr, getPred_fromProb, getMetrics, loadModelData, getCorr, getGroundTruth, getURange
from AnalysisFuncs import getResiduals, plotDistributionProbs, plotCorr_w_Unobs
from AnalysisFuncs import saveFile, loadFile
from AnalysisFuncs import hypertune


In [8]:
train_X = loadFile(osp.join(processed_data_path,'predict_T'), '/train_X.npy')
train_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/train_y_T.npy')
val_X = loadFile(osp.join(processed_data_path,'predict_T'), '/val_X.npy')
val_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/val_y_T.npy')
train_cross_val_X = loadFile(osp.join(processed_data_path,'predict_T'), '/train_cross_val_X.npy')
train_cross_val_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/train_cross_val_y_T.npy')
test_X = loadFile(osp.join(processed_data_path,'predict_T'), '/test_X.npy')
test_y_T = loadFile(osp.join(processed_data_path,'predict_T'), '/test_y_T.npy')

train_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/train_X_D_given_T.npy')
train_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/train_y_D_given_T.npy')
val_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/val_X_D_given_T.npy')
val_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/val_y_D_given_T.npy')
train_cross_val_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), 
                                       '/train_cross_val_X_D_given_T.npy')
train_cross_val_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), 
                                       '/train_cross_val_y_D_given_T.npy')
test_X_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/test_X_D_given_T.npy')
test_y_D_given_T = loadFile(osp.join(processed_data_path,'predict_D_given_T'), '/test_y_D_given_T.npy')



train_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/train_y_D_and_T.npy')
val_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/val_y_D_and_T.npy')
train_cross_val_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), 
                                       '/train_cross_val_y_D_and_T.npy')
test_y_D_and_T = loadFile(osp.join(processed_data_path,'predict_D_and_T'), '/test_y_D_and_T.npy')


In [10]:
# load models
LR_model=LogisticRegression()
LGBM_model = LGBMClassifier()


In [ ]:
#hypertune with LR model
params_dict_LR = dict(C=stats.uniform(loc=0, scale=4), penalty=['l2', 'l1'], solver=['liblinear', 'saga'])
bst_params_T_LR = hypertune(LR_model, params_dict_LR, train_cross_val_X.astype(float), train_cross_val_y_T, cv=5)
bst_params_D_given_T_LR = hypertune(LR_model, params_dict_LR, train_cross_val_X_D_given_T.astype(float),
                                    train_cross_val_y_D_given_T, cv=5)
bst_params_D_and_T_LR = hypertune(LR_model, params_dict_LR, train_cross_val_X.astype(float), 
                                  train_cross_val_y_D_and_T, cv=5)


In [26]:
# save the bst params dict
saveFile(osp.join(processed_data_path, 'predict_T'), bst_params_T_LR, 'bst_params_T_LR.pkl')
saveFile(osp.join(processed_data_path, 'predict_D_given_T'), bst_params_D_given_T_LR, 'bst_params_D_given_T_LR.pkl')
saveFile(osp.join(processed_data_path, 'predict_D_and_T'), bst_params_D_and_T_LR, 'bst_params_D_and_T_LR.pkl')


In [21]:
assert np.isnan(train_cross_val_X.astype(float)).sum()==0
assert np.isnan(train_cross_val_y_T).sum()==0
assert np.isnan(train_cross_val_X_D_given_T.astype(float)).sum()==0
assert np.isnan(train_cross_val_y_D_given_T).sum()==0
assert np.isnan(train_cross_val_y_D_and_T).sum()==0

In [24]:
#hypertune with LGBM model
params_dict_LGBM = dict(max_depth=[3,4,5,6,7], 
                        feature_fraction=stats.uniform(loc=0.5, scale=0.5), 
                        bagging_fraction=stats.uniform(loc=0.7, scale=0.3),
                        min_child_samples=[10,20,30,50])
bst_params_T_LGBM = hypertune(LGBM_model, params_dict_LGBM, train_cross_val_X.astype(float), train_cross_val_y_T, cv=5)
bst_params_D_given_T_LGBM = hypertune(LGBM_model, params_dict_LGBM, train_cross_val_X_D_given_T.astype(float),
                                    train_cross_val_y_D_given_T, cv=5)
bst_params_D_and_T_LGBM = hypertune(LGBM_model, params_dict_LGBM, train_cross_val_X.astype(float), train_cross_val_y_D_and_T, cv=5)


[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Cur

[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Cur

[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8646440511781974
[LightGBM] [Warning] feature_fraction is set=0.8575946831862098, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8575946831862098
[LightGBM] [Warning] bagging_fraction is set=0.8646440511781974, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.7060655192320977, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7060655192320977
[LightGBM] [Warning] feature_fraction is set=0.916309922773969, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.916309922773969
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8420824135821131
[LightGBM] [Warning] feature_fraction is set=0.9004553759898222, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9004553759898222
[LightGBM] [Warning] bagging_fraction is set=0.8420824135821131, subsample=1.0 will be ignored. Cur